# Lecture 3 - 14.06.2021

# Exercise: Polarimetric RGB visualization

* Acquisition: Nkok (Gabon), DLR's F-SAR, L-band

* Path to images: ./data/02-polsar/

* SLC (single-look complex) images:
    * HH: slc_16afrisr0107_Lhh_tcal_test.rat
    * HV: slc_16afrisr0107_Lhv_tcal_test.rat
    * VH: slc_16afrisr0107_Lvh_tcal_test.rat
    * VV: slc_16afrisr0107_Lvh_tcal_test.rat


In [6]:
%matplotlib widget

# import useful libraries, functions, and modules

import sys
sys.path.append('/projects/src/')

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import filters
from ste_io import *

## Exercise 1

Create RGB composite images using 

* Lexicographic basis: R = HH - G = HV - V = VV 
* Pauli basis:         R = HH-VV - G = 2HV - V = HH+VV

In [7]:
# path 2 images
path = './data/02-polsar/'

# looks
looksr = 5
looksa = 5

# open ...
slchh = rrat(path + 'slc_16afrisr0107_Lhh_tcal_test.rat', block = [2500, 16000, 0, 2000])
slcvv = rrat(path + 'slc_16afrisr0107_Lvv_tcal_test.rat', block = [2500, 16000, 0, 2000])
slchv = rrat(path + 'slc_16afrisr0107_Lhv_tcal_test.rat', block = [2500, 16000, 0, 2000])
# slcvh = rrat(path + 'slc_16afrisr0107_Lvh_tcal_test.rat')


In [8]:
# plot 'single-look'
plt.figure( figsize=(15, 15) )

plt.subplot(1, 2, 1)
plt.imshow(np.abs(slchh), vmin = 0, vmax = 3*np.mean(np.abs(slchh)), cmap = 'gray', aspect = 'auto', interpolation = 'nearest')
plt.title('Single-look')

# plot 'multi-look'
amphh = np.abs(slchh)**2
amphh = np.sqrt(filters.uniform_filter(amphh, [looksa,looksr]))

plt.subplot(1, 2, 2)
plt.imshow(np.abs(amphh), vmin = 0, vmax = 3*np.mean(np.abs(amphh)), cmap = 'gray', aspect = 'auto', interpolation = 'nearest')
plt.title('Multilook')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Multilook')

In [9]:
# Make multilooks for VV and HV

ampvv = np.abs(slcvv)**2
ampvv = np.sqrt(filters.uniform_filter(ampvv, [looksa,looksr]))

amphv = np.abs(slchv)**2
amphv = np.sqrt(filters.uniform_filter(amphv, [looksa,looksr]))

# Lexicographic basis - prepare RGB
dimaz = slchh.shape[0]
dimrg = slchh.shape[1]
rgb_lex = np.zeros((dimaz, dimrg, 3), 'float32')

rgb_lex[:, :, 0] = np.clip(amphh, 0, 2.5*np.mean(amphh))     # red
rgb_lex[:, :, 1] = np.clip(amphv, 0, 2.5*np.mean(amphv))     # green
rgb_lex[:, :, 2] = np.clip(ampvv, 0, 2.5*np.mean(ampvv))     # blue

rgb_lex[:, :, 0] = rgb_lex[:, :, 0] / np.max(rgb_lex[:, :, 0])     # red
rgb_lex[:, :, 1] = rgb_lex[:, :, 1] / np.max(rgb_lex[:, :, 1])     # green
rgb_lex[:, :, 2] = rgb_lex[:, :, 2] / np.max(rgb_lex[:, :, 2])     # blue

# kill variables not needed
del amphh
del amphv
del ampvv

# make now multilooks for pauli
pauli1 = np.abs(slchh + slcvv)**2
pauli1 = np.sqrt(filters.uniform_filter(pauli1, [looksa,looksr]))
pauli2 = np.abs(slchh - slcvv)**2
pauli2 = np.sqrt(filters.uniform_filter(pauli2, [looksa,looksr]))
pauli3 = np.abs(2*slchv)**2
pauli3 = np.sqrt(filters.uniform_filter(pauli3, [looksa,looksr]))

# Pauli basis - prepare RGB
rgb_pauli = np.zeros((dimaz, dimrg, 3), 'float32')

rgb_pauli[:, :, 0] = np.clip(pauli2, 0, 2.5*np.mean(pauli2))     # red
rgb_pauli[:, :, 1] = np.clip(pauli3, 0, 2.5*np.mean(pauli3))     # green
rgb_pauli[:, :, 2] = np.clip(pauli1, 0, 2.5*np.mean(pauli1))     # blue

rgb_pauli[:, :, 0] = rgb_pauli[:, :, 0] / np.max(rgb_pauli[:, :, 0])     # red
rgb_pauli[:, :, 1] = rgb_pauli[:, :, 1] / np.max(rgb_pauli[:, :, 1])     # green
rgb_pauli[:, :, 2] = rgb_pauli[:, :, 2] / np.max(rgb_pauli[:, :, 2])     # blue

# kill variables not needed
del pauli1
del pauli2
del pauli3

# now plot RGB
plt.figure( figsize = (15, 15) )

plt.subplot(1, 2, 1)
plt.imshow(rgb_lex, aspect = 'auto', interpolation = 'nearest')
plt.title('Lexicographic')

plt.subplot(1, 2, 2)
plt.imshow(rgb_pauli, aspect = 'auto', interpolation = 'nearest')
plt.title('Pauli')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Pauli')

## Exercise 2

Calculate polarimetric coherences and phase differences between
* HH and HV
* VV and VH
* HH and VV

In [10]:
del rgb_lex
del rgb_pauli

# HH-HV

amphh = np.abs(slchh)**2
amphh = np.sqrt(filters.uniform_filter(amphh, [looksa,looksr]))

amphv = np.abs(slchv)**2
amphv = np.sqrt(filters.uniform_filter(amphv, [looksa,looksr]))

corrhhhv = filters.uniform_filter(np.real(slchh*np.conj(slchv)), [looksa,looksr]) + 1j* \
                filters.uniform_filter(np.imag(slchh*np.conj(slchv)), [looksa,looksr])

gamma_hhhv = corrhhhv / amphh / amphv

# now plot RGB
plt.figure( figsize = (15, 15) )

plt.subplot(1, 2, 1)
plt.imshow(np.abs(gamma_hhhv), vmin = 0, vmax = 1, aspect = 'auto', cmap = 'gray', interpolation = 'nearest')
plt.title('HH-HV Abs.')

plt.subplot(1, 2, 2)
plt.imshow(np.angle(slchh*np.conj(slchv)), vmin = -np.pi, vmax = np.pi, aspect = 'auto', cmap = 'jet', interpolation = 'nearest')
plt.title('HH-HV Pha.')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'HH-HV Pha.')

## To be continued ...